# Import libraries, data and models

In [34]:
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np

# Load cleaned data
movies = pd.read_pickle('movies_df.pkl')
ratings = pd.read_pickle('ratings_df.pkl')

# Load saved TF-IDF model and vectorizer
vectorizer = joblib.load('tfidf_vectorizer.pkl')
tfidf_matrix = joblib.load('tfidf_matrix.pkl')

# Load BERT embeddings
movies_embeddings = joblib.load('movies_embeddings.pkl')

In [35]:
# Load ratings data
# Replace with appropriate paths or methods to read the data
ratings_train = pd.read_csv('archive/ml-100k/ua.base', delimiter='\t', names=['userId', 'movieId', 'rating', 'timestamp'])
ratings_test = pd.read_csv('archive/ml-100k/ua.test', delimiter='\t', names=['userId', 'movieId', 'rating', 'timestamp'])

# Merge movie info with ratings for both train and test data
train_data = pd.merge(ratings_train, movies[['movieId', 'title', 'combined_info']], on='movieId')
test_data = pd.merge(ratings_test, movies[['movieId', 'title', 'combined_info']], on='movieId')

# TF-IDF model

In [36]:
def recommend_for_user_tfidf(user_id, train_data, top_n=5):
    user_ratings = train_data[train_data['userId'] == user_id]
    if user_ratings.empty:
        return pd.DataFrame()  # No recommendations if user has no ratings in the training data

    # Get user profile
    user_profile = user_ratings[['movieId', 'rating']].copy()
    user_profile = user_profile.merge(movies[['movieId', 'combined_info']], on='movieId')
    user_profile_matrix = vectorizer.transform(user_profile['combined_info'])
    user_profile_vector = np.asarray(np.mean(user_profile_matrix.toarray(), axis=0))  # Convert matrix to array
    
    # Compute similarity between the user profile and all movie vectors
    similarities = cosine_similarity(user_profile_vector.reshape(1, -1), tfidf_matrix)[0]
    
    # Add similarities to the original data
    movies['similarity'] = similarities
    recommendations = movies[['movieId', 'title', 'similarity']].sort_values(by='similarity', ascending=False)
    
    # Return top N recommendations
    return recommendations.head(top_n)

# BERT model

In [37]:
# Define recommendation function
def recommend_for_user_bert(user_id, train_data, top_n=5):
    user_ratings = train_data[train_data['userId'] == user_id]
    if user_ratings.empty:
        return pd.DataFrame()  # No recommendations if user has no ratings in the training data

    # Get user profile
    user_profile = user_ratings[['movieId', 'rating']].copy()
    user_profile = user_profile.merge(movies_embeddings[['movieId', 'embedding']], on='movieId')
    user_profile_vector = np.mean(user_profile['embedding'].tolist(), axis=0)
    
    # Compute similarity between the user profile and all movie vectors
    all_movie_vectors = np.array(movies_embeddings['embedding'].tolist())
    similarities = cosine_similarity([user_profile_vector], all_movie_vectors)[0]
    
    # Add similarities to the original data
    movies['similarity'] = similarities
    recommendations = movies[['movieId', 'title', 'similarity']].sort_values(by='similarity', ascending=False)
    
    # Return top N recommendations
    return recommendations.head(top_n)

# Evaluation functions

In [53]:
def evaluate_tfidf(test_data, train_data, top_n=5):
    y_true = []
    y_pred = []

    for user_id in test_data['userId'].unique():
        recommendations = recommend_for_user_tfidf(user_id, train_data, top_n)
        if recommendations.empty:
            continue  # Skip users with no recommendations
        
        recommended_movie_ids = set(recommendations['movieId'])
        actual_movie_ids = set(test_data[test_data['userId'] == user_id]['movieId'])
        
        # Prepare ground truth and predictions
        # True = 1 for relevant movies (actual_movie_ids), 0 otherwise
        for movie_id in actual_movie_ids:
            y_true.append(1)
            y_pred.append(1 if movie_id in recommended_movie_ids else 0)
        
        # Handle movies in recommendations that are not in actual_movie_ids
        for movie_id in recommended_movie_ids:
            if movie_id not in actual_movie_ids:
                y_true.append(0)
                y_pred.append(1)
        
    # Ensure y_true and y_pred have the same length
    if len(y_true) == 0 or len(y_pred) == 0:
        return None, None, None  # Return None if there are no valid recommendations
    
    # Calculate evaluation metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    return precision, recall, f1

In [54]:
def evaluate_bert(test_data, train_data, top_n=5):
    y_true = []
    y_pred = []

    for user_id in test_data['userId'].unique():
        recommendations = recommend_for_user_bert(user_id, train_data, top_n)
        if recommendations.empty:
            continue  # Skip users with no recommendations
        
        recommended_movie_ids = set(recommendations['movieId'])
        actual_movie_ids = set(test_data[test_data['userId'] == user_id]['movieId'])
        
        # Prepare ground truth and predictions
        # True = 1 for relevant movies (actual_movie_ids), 0 otherwise
        for movie_id in actual_movie_ids:
            y_true.append(1)
            y_pred.append(1 if movie_id in recommended_movie_ids else 0)
        
        # Handle movies in recommendations that are not in actual_movie_ids
        for movie_id in recommended_movie_ids:
            if movie_id not in actual_movie_ids:
                y_true.append(0)
                y_pred.append(1)
        
    # Ensure y_true and y_pred have the same length
    if len(y_true) == 0 or len(y_pred) == 0:
        return None, None, None  # Return None if there are no valid recommendations
    
    # Calculate evaluation metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    return precision, recall, f1

# Recommendations

In [55]:
recommend_for_user_tfidf(1, train_data, top_n=5)

,movieId,title,similarity
1024,1025,Fire Down Below (1997),0.506325
830,831,Escape from L.A. (1996),0.469919
1101,1102,Two Much (1996),0.422284
1047,1048,She's the One (1996),0.422284
357,358,Spawn (1997),0.420208


In [56]:
recommend_for_user_bert(1, train_data, top_n=5)

,movieId,title,similarity
338,339,Mad City (1997),0.920737
627,628,Sleepers (1996),0.918142
788,789,Swimming with Sharks (1995),0.916811
1459,1460,Sleepover (1995),0.915964
544,545,Vampire in Brooklyn (1995),0.913734


# Evaluation

In [59]:
precision_tfidf, recall_tfidf, f1_tfidf = evaluate_tfidf(test_data, train_data, top_n = 50)
print(f"TF-IDF Model - Average Precision: {precision_tfidf}")
print(f"TF-IDF Model - Average Recall: {recall_tfidf}")
print(f"TF-IDF Model - Average F1 Score: {f1_tfidf}")

precision_bert, recall_bert, f1_bert = evaluate_bert(test_data, train_data, top_n = 50)
print(f"BERT Model - Average Precision: {precision_bert}")
print(f"BERT Model - Average Recall: {recall_bert}")
print(f"BERT Model - Average F1 Score: {f1_bert}")

TF-IDF Model - Average Precision: 0.02462354188759279
TF-IDF Model - Average Recall: 0.12311770943796395
TF-IDF Model - Average F1 Score: 0.04103923647932131
BERT Model - Average Precision: 0.012873806998939554
BERT Model - Average Recall: 0.06436903499469777
BERT Model - Average F1 Score: 0.02145634499823259


# RMSE

In [60]:
def predict_rating_tfidf(user_id, movie_id, train_data):
    # Get user profile based on ratings
    user_ratings = train_data[train_data['userId'] == user_id]
    if user_ratings.empty:
        return None  # Return None if user has no ratings

    # Compute user's average profile
    user_profile = user_ratings[['movieId', 'rating']].merge(movies[['movieId', 'combined_info']], on='movieId')
    user_profile_matrix = vectorizer.transform(user_profile['combined_info'])
    user_profile_vector = np.mean(user_profile_matrix.toarray(), axis=0)

    # Get the vector for the movie we're predicting
    movie_vector = tfidf_matrix[movies['movieId'] == movie_id].toarray().flatten()

    # Compute similarity between user's profile and the movie
    similarity = cosine_similarity(user_profile_vector.reshape(1, -1), movie_vector.reshape(1, -1))[0][0]

    # Predict rating as a weighted sum of user's past ratings
    if len(user_ratings) > 0:
        weighted_ratings = user_ratings['rating'].to_numpy() * similarity
        predicted_rating = np.mean(weighted_ratings)
    else:
        predicted_rating = similarity * 5  # If no prior ratings, use similarity as a proxy

    return predicted_rating

In [61]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluate_rmse_tfidf(test_data, train_data):
    actual_ratings = []
    predicted_ratings = []
    
    for _, row in test_data.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        actual_rating = row['rating']
        
        predicted_rating = predict_rating_tfidf(user_id, movie_id, train_data)
        if predicted_rating is not None:
            actual_ratings.append(actual_rating)
            predicted_ratings.append(predicted_rating)
    
    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
    return rmse


In [62]:
def predict_rating_bert(user_id, movie_id, train_data):
    # Get user profile based on ratings
    user_ratings = train_data[train_data['userId'] == user_id]
    if user_ratings.empty:
        return None  # Return None if user has no ratings

    # Compute user's average profile
    user_profile = user_ratings[['movieId', 'rating']].merge(movies_embeddings[['movieId', 'embedding']], on='movieId')
    user_profile_vector = np.mean(np.array(user_profile['embedding'].tolist()), axis=0)

    # Get the vector for the movie we're predicting
    movie_vector = np.array(movies_embeddings[movies_embeddings['movieId'] == movie_id]['embedding'].tolist())[0]

    # Compute similarity between user's profile and the movie
    similarity = cosine_similarity([user_profile_vector], [movie_vector])[0][0]

    # Predict rating as a weighted sum of user's past ratings
    if len(user_ratings) > 0:
        weighted_ratings = user_ratings['rating'].to_numpy() * similarity
        predicted_rating = np.mean(weighted_ratings)
    else:
        predicted_rating = similarity * 5  # If no prior ratings, use similarity as a proxy

    return predicted_rating


In [63]:
def evaluate_rmse_bert(test_data, train_data):
    actual_ratings = []
    predicted_ratings = []
    
    for _, row in test_data.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        actual_rating = row['rating']
        
        predicted_rating = predict_rating_bert(user_id, movie_id, train_data)
        if predicted_rating is not None:
            actual_ratings.append(actual_rating)
            predicted_ratings.append(predicted_rating)
    
    # Compute RMSE
    rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
    return rmse


In [64]:
# Evaluate TF-IDF model
rmse_tfidf = evaluate_rmse_tfidf(test_data, train_data)
print(f'TF-IDF Model RMSE: {rmse_tfidf}')

# Evaluate BERT model
rmse_bert = evaluate_rmse_bert(test_data, train_data)
print(f'BERT Model RMSE: {rmse_bert}')


TF-IDF Model RMSE: 3.1510211794973704
BERT Model RMSE: 1.1801095335094431


# Hit rate

In [65]:
def evaluate_hit_rate_tfidf(test_data, train_data, top_n=5, positive_threshold=4):
    hits = 0
    total_users = 0

    for user_id in test_data['userId'].unique():
        # Get top-N recommendations
        recommendations = recommend_for_user_tfidf(user_id, train_data, top_n)
        
        # Get the user's actual positively rated movies
        user_ratings = test_data[(test_data['userId'] == user_id) & (test_data['rating'] >= positive_threshold)]
        actual_positive_movie_ids = set(user_ratings['movieId'])
        
        # Check if any of the recommended movies are in the actual positive ratings
        recommended_movie_ids = set(recommendations['movieId'])
        hit = len(recommended_movie_ids.intersection(actual_positive_movie_ids)) > 0
        
        if hit:
            hits += 1
        total_users += 1

    hit_rate = hits / total_users if total_users > 0 else 0
    return hit_rate

# Evaluate Hit Rate for TF-IDF model
hit_rate_tfidf = evaluate_hit_rate_tfidf(test_data, train_data, top_n=5)
print(f"TF-IDF Model Hit Rate: {hit_rate_tfidf}")

TF-IDF Model Hit Rate: 0.029692470837751856


In [68]:
def evaluate_hit_rate_bert(test_data, train_data, top_n=5, positive_threshold=4):
    hits = 0
    total_users = 0

    for user_id in test_data['userId'].unique():
        # Get top-N recommendations
        recommendations = recommend_for_user_bert(user_id, train_data, top_n)
        
        # Get the user's actual positively rated movies
        user_ratings = test_data[(test_data['userId'] == user_id) & (test_data['rating'] >= positive_threshold)]
        actual_positive_movie_ids = set(user_ratings['movieId'])
        
        # Check if any of the recommended movies are in the actual positive ratings
        recommended_movie_ids = set(recommendations['movieId'])
        hit = len(recommended_movie_ids.intersection(actual_positive_movie_ids)) > 0
        
        if hit:
            hits += 1
        total_users += 1

    hit_rate = hits / total_users if total_users > 0 else 0
    return hit_rate

# Evaluate Hit Rate for TF-IDF model
hit_rate_bert = evaluate_hit_rate_bert(test_data, train_data, top_n=5)
print(f"BERT Model Hit Rate: {hit_rate_bert}")

BERT Model Hit Rate: 0.030752916224814422
